クロスデバイス　企業IPとの併用

In [1]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

8663	株式会社スキマビ

In [2]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

190530


LS0063_従業員30人以上

In [6]:
seed_table = 'segment.ya_landscape_ls0063'
xd_table = 'segment.drawbridge_device'
segment_id = 'ls0063'
uid_table = f"segment.ya_xd_{segment_id}_{today_yymmdd}"
print(uid_table)

segment.ya_xd_ls0063_190530


In [7]:
%%slack_notify @yusuke_akada
#XD拡張

#f-string
query = (
    f"""
    select 
    distinct xd2.smn_uid
    ,sd.company_name
    from {seed_table} sd
    left join {xd_table} xd
    on sd.smn_uid = xd.smn_uid
    left join {xd_table} xd2
    on xd.drawbridge_consumer_id = xd2.drawbridge_consumer_id
    where xd2.device in ('iphone'
    ,'android_phone'
    ,'windows_phone'
    ,'windows_computer'
    ,'linux_computer'
    ,'mac'
    ,'ipad'
    ,'android_tablet'
    ,'windows_tablet') 
    and sd.smn_uid <> xd2.smn_uid
    
    
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_fix = ir.sql_to_pandas(query)
    ir.upload_dataframe(uid_table, df_fix, ['bigint','string'])

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [8]:
df_fix.shape[0]

6354762

In [9]:
print(df_fix.dtypes)

smn_uid          int64
company_name    object
dtype: object
